In [97]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import numpy as np 
import pandas as pd

In [99]:
dataset_path = "./dataset.csv"

df = pd.read_csv(
    dataset_path,
    # dtype is specified to prevent a DtypeWarning
    dtype={
        "full_query": str,
        "label": int,
        "statement_type": str,
        "query_template_id": str,
        "attack_payload": str,
        "attack_id": str,
        "attack_technique": str,
        "attack_desc": str,
        "split": str,
        "sqlmap_status" : str,
        "attack_stage" : str,
        "tamper_method" : str

    },
)

display(df)


,full_query,label,attack_payload,attack_desc,attack_stage,tamper_method,sqlmap_status,statement_type,query_template_id,attack_id,attack_technique,split
0,"UPDATE regions SET keywords = ""Kärnten"" WHERE ...",0,NaN,NaN,NaN,NaN,NaN,update,airport-U9,NaN,NaN,test
1,"INSERT INTO regions (code, local_code, name, c...",1,NaN,NaN,recon,space2mysqlblank,NaN,insert,airport-I5,error-5,error,test
2,"SELECT 'Airport Frequency' as source_type, af....",0,NaN,NaN,NaN,NaN,NaN,select,airport-S13,NaN,NaN,test
3,"INSERT INTO runways (airport_ref, airport_iden...",1,NaN,NaN,recon,space2comment,NaN,insert,airport-I9,error-7,error,test
4,"INSERT INTO airport (ident, type, name, latitu...",0,NaN,NaN,NaN,NaN,NaN,insert,airport-I2,NaN,NaN,test
...,...,...,...,...,...,...,...,...,...,...,...,...
39681,"INSERT INTO countries (code, name, continent, ...",0,NaN,NaN,NaN,NaN,NaN,insert,airport-I6,NaN,NaN,test
39682,"INSERT INTO regions (code, name, continent, is...",0,NaN,NaN,NaN,NaN,NaN,insert,airport-I4,NaN,NaN,test
39683,"INSERT INTO regions (code, name, continent, is...",0,NaN,NaN,NaN,NaN,NaN,insert,airport-I4,NaN,NaN,test
39684,"INSERT INTO regions (code, local_code, name, c...",0,NaN,NaN,NaN,NaN,NaN,insert,airport-I5,NaN,NaN,test


In [100]:
def display_attack_normal_per_set(df : pd.DataFrame):
    df_train_atk = df[(df["label"] == 1) & (df["split"] == "train")]  
    df_train_normal = df[(df["label"] == 0) & (df["split"] == "train")]  

    df_test_atk = df[(df["label"] == 1) & (df["split"] == "test")]  
    df_test_normal = df[(df["label"] == 0) & (df["split"] == "test")]  

    print(f"Number of attacks in train: {df_train_atk.shape[0]}")
    print(f"Number of normal in train: {df_train_normal.shape[0]}")
    print(f"Number of attacks in test: {df_test_atk.shape[0]}")
    print(f"Number of normal in test: {df_test_normal.shape[0]}")

display_attack_normal_per_set(df)

Number of attacks in train: 4296
Number of normal in train: 5525
Number of attacks in test: 2490
Number of normal in test: 27375


In [101]:
def display_ids_per_set(df: pd.DataFrame): 
    df_train = df[df["split"] == "train"]
    df_test = df[df["split"] == "test"]

    tids_train  = df_train['query_template_id'].unique()
    tids_test = df_test['query_template_id'].unique()
    tids_complement = set(tids_test) - set(tids_train)
    print(f"Template IDS in df train: {len(tids_train)}, {tids_train}")
    print(f"Template IDS in df test:   {len(tids_test)}, {tids_test}")
    print(f"Template IDs in train but NOT in test: {len(tids_complement)}, {sorted(tids_complement)}")
    
display_ids_per_set(df)

Template IDS in df train: 42, ['airport-S3' 'airport-I3' 'airport-I5' 'airport-S8' 'airport-D2'
 'airport-I2' 'airport-S10' 'airport-I6' 'airport-U3' 'airport-S17'
 'airport-S4' 'airport-S15' 'airport-S2' 'airport-S9' 'airport-S1'
 'airport-D8' 'airport-S7' 'airport-U4' 'airport-S5' 'airport-I9'
 'airport-D1' 'airport-S12' 'airport-S13' 'airport-S11' 'airport-S6'
 'airport-D5' 'airport-U1' 'airport-I7' 'airport-D6' 'airport-I8'
 'airport-S16' 'airport-U8' 'airport-U9' 'airport-I1' 'airport-U2'
 'airport-D7' 'airport-U7' 'airport-U6' 'airport-U5' 'airport-D4'
 'airport-D3' 'airport-I10']
Template IDS in df test:   44, ['airport-U9' 'airport-I5' 'airport-S13' 'airport-I9' 'airport-I2'
 'airport-S8' 'airport-S14' 'airport-U8' 'airport-U3' 'airport-S2'
 'airport-S1' 'airport-I4' 'airport-S3' 'airport-S15' 'airport-I3'
 'airport-S17' 'airport-S10' 'airport-S11' 'airport-S4' 'airport-U4'
 'airport-D2' 'airport-D6' 'airport-S5' 'airport-D8' 'airport-I10'
 'airport-S16' 'airport-S6' 'airport-S